In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
# Define paths and parameters
classes_path = "C:\\Users\\Dimple\\OneDrive\\Desktop\\PBL\\dataset\\train"
class_names = ['\\Grape___Black_rot', '\\Grape___Esca_(Black_Measles)', '\\Grape___healthy', '\\Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']
num_classes = len(class_names)
img_size = (224, 224)

In [2]:
# Function to preprocess images
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.resnet.preprocess_input(img_array)  # Preprocess for ResNet
    return img_array
print(f'class names: {class_names}\nclasses number: {num_classes}')

class names: ['\\Grape___Black_rot', '\\Grape___Esca_(Black_Measles)', '\\Grape___healthy', '\\Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']
classes number: 4


In [3]:
images = []
labels = []
print('images:\n')
for cl in class_names:
    print(cl, end=' -> ')
    for img in os.listdir(classes_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        img_path = os.path.join(classes_path + cl, img)
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        images.append(img_array)
    print('done')

images:

\Grape___Black_rot -> done
\Grape___Esca_(Black_Measles) -> done
\Grape___healthy -> done
\Grape___Leaf_blight_(Isariopsis_Leaf_Spot) -> done


In [4]:
from matplotlib import pyplot as plt
from random import randint
labels = np.asarray(labels)
images = np.asarray(images)

print(f'\n\nlabels shape: {labels.shape}')
print(f'images shape: {images.shape}')




labels shape: (7222, 4)
images shape: (7222, 224, 224, 3)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define base models
base_model_densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

In [6]:
# Add classification layers on top of each base model
x_densenet = base_model_densenet.output
x_densenet = GlobalAveragePooling2D()(x_densenet)
x_densenet = Dense(512, activation='relu')(x_densenet)
x_densenet = Dense(256, activation='relu')(x_densenet)
predictions_densenet = Dense(num_classes, activation='softmax')(x_densenet)

x_resnet = base_model_resnet.output
x_resnet = GlobalAveragePooling2D()(x_resnet)
x_resnet = Dense(512, activation='relu')(x_resnet)
x_resnet = Dense(256, activation='relu')(x_resnet)
predictions_resnet = Dense(num_classes, activation='softmax')(x_resnet)

In [7]:
# Create models
densenet_model = Model(inputs=base_model_densenet.input, outputs=predictions_densenet)
resnet_model = Model(inputs=base_model_resnet.input, outputs=predictions_resnet)

In [8]:
# Freeze base layers
for layer in base_model_densenet.layers:
    layer.trainable = False
for layer in base_model_resnet.layers:
    layer.trainable = False

In [9]:
# Compile models
densenet_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train models
densenet_model.fit(X_train, y_train, epochs=1, verbose=1, validation_data=(X_val, y_val))
resnet_model.fit(X_train, y_train, epochs=1, verbose=1, validation_data=(X_val, y_val))
densenet_model.save("densenet_model_1epoch.h5")
resnet_model.save("resenet_model_1epoch.h5")
# Evaluate accuracy
accuracy = np.mean(np.argmax(y_val, axis=1) == np.argmax(ensemble_predictions, axis=1))
print("Ensemble Accuracy:",accuracy)



204/204 [==============================] - 356s 2s/step - loss: 0.3805 - accuracy: 0.8840 - val_loss: 0.3898 - val_accuracy: 0.8313


MemoryError: Unable to allocate 3.64 GiB for an array with shape (6499, 224, 224, 3) and data type float32

In [ ]:
from sklearn.metrics import confusion_matrix
# Get the predicted classes
predicted_classes = np.argmax(ensemble_predictions, axis=1)

# Get the true classes
true_classes = np.argmax(y_val, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

print("Confusion Matrix:")
print(conf_matrix)